# Section 3: For-Loops and Iterables
In this section, we will introduce the essential "for-loop" contol flow paradigm along with the formal definition of an "iterable". Additionally, we will encounter some of the most essential features of Python that sets it apart as a language that is both easy to read and write. These include:
- iterable unpacking
- iterable enumerations
- generators
- list comprehensions

The utility of these items cannot be understated. Moving forward, you will likely find use for these concepts in nearly every piece of Python code that you write!

## For-Loops
A "for-loop" allows you to iterate over a collection of items, and execute a block of code once for each iteration. The general syntax for a "for-loop" is:

```
for <var> in <iterable>:
    block of code
```

Where `<var>` is any valid variable-identifier and `<iterable>` is any **iterable**.

<div class="alert alert-block alert-info"> 
**Definition**: An **iterable** is any Python object capable of returning its members one at a time. Common examples of iterables include lists, tuples, strings, and dictionaries. Formally, an iterable is any Python object with an `__iter__()` method or with a `__getitem__()` method that implements `Sequence` semantics.
</div>
The for-loop behaves as follows:
- Request the next member of the iterable.
- If the iterable is empty, exit the for-loop without running the enclosed code block.
- If the iterable did produce a member, assign that member to `<var>` (if `<var>` was not previously defined, it becomes defined). 
- Execute the enclosed block of code.
- Go back to the first step.

To be concrete, let's consider the example:
```python
total = 0
for item in [1, 3, 5]:
    total = total + item

print(total)  # `total` has the value 1 + 3 + 5 = 9
# `item` is still defined here, and holds the value 5
```

This code will perform the following steps:
1. Define the variable `total`, and assign it the value `0`
2. Iterate on the list, producing value `1`, define the variable `item` and assign it the value `1`
3. Assign `total` the value `0 + 1`
4. Iterate on the list, producing the value `3` and assigning it to `item`
5. Assign `total` the value `1 + 3`
6. Iterate on the list, producing the value `5` and assigning it to `item`
7. Assign `total` the value `4 + 5`
8. Iterate on the list. Having reached its end, a `StopIteration` signal it raised by the list, and the for-loop sequence is exited.
9. Print the value of `total` (9)

#### Potential Pitfall
Note that the variable `item` will persist after the for-loop block is exited. It will reference the last value from the for-loop iteration (in this case `item` has the value 5). That being said, **you should not write code that depends on the iterate-variable, outside of the context of the for-loop**. In the case that you try to loop over an *empty* iterable, the iterate-variable is never defined:

```python
for x in []:         # the iterable is empty - the iterate-variable `x` will not be defined
    print("Hello?")  # this code is never executed
print(x)             # raises an error because `x` was never defined
```

Because we are attempting to iterate over an empty list, `StopIteration` is raised immediately - before the variable `x` is even defined. Thus the code enclosed within the for-loop is never reached, and the subsequent `print(x)` statement will raise a `NameError`, because `x` was never defined!

## "Unpacking" Iterables
Suppose that you have three values stored in a list, and that you want to assign a distinct variable to each value. Given the lessons that we have covered thus far, you would likely write the following code:

```python
# simple script for assigning variables to contents of list
>>> my_list = [7, 9, 11]

>>> x = my_list[0]
>>> y = my_list[1]
>>> z = my_list[2]
```

Python provides an extremely useful functionality, known as **iterable unpacking**, which allows us to write the simple, elegant code:

```python
# assigning variables to contents of a list using iterable unpacking
>>> my_list = [7, 9, 11]

>>> x, y, z = my_list
>>> print(x, y, z)
7 9 11
```

That is, the Python interpreter "sees" the pattern of variables to the left of the assignment, and will "unpack" the iterable (which happens to be a list in this instance). It may not seem it from this example, but this is an *extremely* useful feature of Python that greatly facilitates the readability of code! 

Iterable unpacking is particularly useful in the context of performing for-loops over iterables-of-iterables. For example, suppose we have a list containing tuples of name-grade pairs:

```python
>>> grades = [("Ashley", 93), ("Brad", 95), ("Cassie", 84)]
```

Recall from the preceding section that if we loop over this list, that the iterate-variable will be assigned to each of these tuples:

```python
for entry in grades:
    print(entry)
```
will print:
```
('Ashley', 93)
('Brad', 95)
('Cassie', 84)
```

It is likely that we will want to work with the student's name and their grade independently (e.g. use the name to access a log, and add the grade-value to our class statistics), thus we will need to index into `entry` twice to assign its contents to two separate variables. However, because each iteration of the for-loop involves an assignment of the form `entry = ("Ashley", 93)`, we can make use of iterable unpacking!

```python
# The first iteration of this for-loop performs
# the unpacking assignment: name, grade = ("Ashley", 93)
for name, grade in grades: 
    # name = "Ashley", grade = 93
    # name = "Brad",   grade = 95
    # name = "Cassie", grade = 84
    print(name)
    print(grade)
    print("\n")
```
prints:
```
Ashley
93

Brad 
95

Cassie 
84
```
This for-loop code is concise and supremely readable. It is highly recommended that you make use of iterable unpacking in such contexts.

Iterable unpacking is not quite as simple as it might seem. What happens if you provide 4 variables to unpack into, but use an iterable containing 10 items? Although what we have covered thus far conveys the most essential use case, it is good to know that [Python provides an even more extensive syntax for unpacking iterables](https://www.python.org/dev/peps/pep-3132/#specification). We will also see that unpacking can be useful when creating and using functions.

<div class="alert alert-block alert-success">
**Takeaway**: Python provides a sleek syntax for "unpacking" the contents of an iterable - assigning each item to its own variable. This allows us to write intuitive, highly-readable code when performing a for-loop over a collection of iterables. 
</div>

## Enumerating Iterables
It is often useful to keep track of the iteration-count of a for-loop. Suppose we want to record all of the positions in a list where the value `None` is stored:

```python
# track which entries of an iterable store the value `None`
none_indices = []
iter_cnt = 0  # manually track iteration-count
for item in [2, None, -10, None, 4, 8]:
    if item is None:
        none_indices.append(iter_cnt)
    iter_cnt = iter_cnt + 1

# `none_indices` now stores: [1, 3]
```

We can make use of the built-in [`enumerate`](https://docs.python.org/3/library/functions.html#enumerate) function so that we don't need to initialize and manually increment the `iter_cnt` variable:

```python
# using `enumerate` function to keep iteration-count
none_indices = []
for iter_cnt, item in enumerate([2, None, -10, None, 4, 8]):  # note the use of iterable unpacking! 
    # iter_cnt = 0, item = 2
    # iter_cnt = 1, item = None
    # iter_cnt = 2, item = -10
    # etc.
    if item is None:
        none_indices.append(iter_cnt)
        
# `none_indices` now stores: [1, 3]
```

In general, the `enumerate` function accepts an iterable and returns a new iterable that produces a tuple of the iteration-count and the corresponding item from the original iterable:
>```
enumerate(<item_0, item_1, ..., item_n>) :: <(0, item_0), (1, item_1), ..., (n, item_n)>

><> represents any iterable (list, tuple, etc)
```


<div class="alert alert-block alert-warning"> 
**Note**: It is important to recognize that the use of `enumerate` function makes for-loops more concise and readable *if it is used in conjunction with iterable unpacking*.
</div>

<div class="alert alert-block alert-success">
**Takeaway**: The built-in [`enumerate`](https://docs.python.org/3/library/functions.html#enumerate) function should be used (in conjuction with iterator unpacking) whenever it is necessary to track the iteration count of a for-loop.   
</div>

## Introducing Generators
Now we introduce an important type of iterable object called a **generator**, which allows use to generate arbitrarily-many items in a sequence of data, without having to store them all in memory at once.  

<div class="alert alert-block alert-info"> 
**Definition**: An **generator** is a special kind of iterable, which stores the instructions for how to *generate* each of its members, in order, along with its current state of iterations. It generates each member only as it is requested via iteration.
</div>

To contrast a generator with more familiar iterables, note that a list readily stores all of its members - you can access any of its contents via indexing. A generator, on the otherhand, *does not store any items*. Instead, it stores the instructions for generating each of its members in sequence, and stores its iteration state - meaning that the generator will know if it has generated its second member, and will thus generate its third member the next time it is iterated on. The whole point of this is that you can use a generator to produce a long sequence of items, without having to store them all in memory.

### `range`: a useful, but goofy generator

In [15]:
gen = (i**2 for i in range(10))

In [26]:
next(gen)

StopIteration: 